In [ ]:
# Import necessary libraries
import time
import datetime
import numpy as np
import pandas as pd
from copy import deepcopy
import glob
import warnings

# Ignore warnings for cleaner output
warnings.filterwarnings("ignore")

# Set pandas options to display all columns
pd.set_option('display.max_columns', None)

# Path to save the resampled data
path_to_save_data = r'D:\PythonProject\Trading\R Data test\Resample Data\5min'

# Path to read original data using glob for multiple files
path = fr'D:\PythonProject\Trading\R Data test\NIfty5002016-01-01to2023-10-23\**'

# Get a list of file paths and create a DataFrame to store them
files_df = pd.DataFrame(glob.glob(path), columns=['File_path'])

# Extract the file name from the file path
files_df['File_Name'] = files_df['File_path'].apply(lambda x: x.split('\\')[-1].split(".")[0])

# Loop through each file path in the DataFrame
for i, s_name in files_df[1:].iterrows():
    file_path = s_name['File_path']  # Get the file path
    ticker = s_name['File_Name']  # Get the file name (ticker symbol)

    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)

    # Combine 'Date' and 'TIME1' columns and convert it to datetime format
    df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['TIME1'], format="%Y-%m-%d %H:%M:%S")
    df.set_index('Datetime', inplace=True)  # Set the Datetime column as the index

    # Resample the data to 5-minute intervals, aggregating OHLCV values
    resample = df.resample("5T").agg({
        "Open": "first",
        "High": "max",
        "Low": "min",
        "CLOSE": "last",
        "VOLUME": "sum"
    }).between_time('09:15:00', '15:25:00').dropna()  # Filter data between market hours and remove NaN values

    resample = resample.reset_index()  # Reset the index
    resample.columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']  # Rename columns for consistency

    # Define the file save path for the resampled data
    file_save_path = f"{path_to_save_data}\\{ticker}.csv"

    # Save the resampled data to a CSV file without index
    resample.to_csv(file_save_path, index=False)
